In [304]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [305]:
df_mum = pd.read_csv("Dataset/Cleaned_Score/mumbai_hotels_scored_cleaned.csv")
df_mum.insert(2, "City", ['Mumbai' for x in range(df_mum.shape[0])], True)
df_del = pd.read_csv("Dataset/Cleaned_Score/delhi_hotels_scored_cleaned.csv")
df_del.insert(2, "City", ['Delhi' for x in range(df_del.shape[0])], True)
df_ban = pd.read_csv("Dataset/Cleaned_Score/bangalore_hotels_scored_cleaned.csv")
df_ban.insert(2, "City", ['Bangalore' for x in range(df_ban.shape[0])], True)
df_che = pd.read_csv("Dataset/Cleaned_Score/chennai_hotels_scored_cleaned.csv")
df_che.insert(2, "City", ['Chennai' for x in range(df_che.shape[0])], True)
df_kol = pd.read_csv("Dataset/Cleaned_Score/kolkata_hotels_scored_cleaned.csv")
df_kol.insert(2, "City", ['Kolkata' for x in range(df_kol.shape[0])], True)

df = df_mum.copy(deep=True)
df = df.append(df_del, ignore_index = True)
df = df.append(df_ban, ignore_index = True)
df = df.append(df_che, ignore_index = True)
df = df.append(df_kol, ignore_index = True)
dum = pd.get_dummies(df.City)
df=df.drop('City', axis=1)
df = pd.concat([df, dum], axis=1)
df

,Name,Latitude,Longitude,Air conditioning,Housekeeping,Private balcony,Private bathrooms,VIP room facilities,Complimentary toiletries,Room service,...,Rating,Number of reviews,Price,Distance Score,Review Score,Bangalore,Chennai,Delhi,Kolkata,Mumbai
0,Lemon Tree Premier-Mumbai International Airport,19.107975,72.882187,1,0,0,0,0,0,1,...,4.5,382,3883,163.06,8.42,0,0,0,0,1
1,The Fern - Goregaon,19.162237,72.857132,1,0,0,0,0,0,1,...,4.5,771,3541,134.39,9.27,0,0,0,0,1
2,"Taj Lands End, Mumbai",19.043695,72.819252,1,1,0,1,1,1,1,...,4.5,4732,5459,208.03,9.50,0,0,0,0,1
3,Grand Hyatt Mumbai Hotel & Residences,19.076893,72.851112,1,1,0,1,1,1,1,...,4.5,4709,5617,201.62,9.50,0,0,0,0,1
4,"Trident, Bandra Kurla, Mumbai",19.066547,72.867554,1,1,0,0,1,0,1,...,4.5,4613,5040,189.10,9.50,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1223,Backpackers Park,22.554041,88.351837,0,0,0,0,0,0,0,...,4.0,10,800,574.08,4.20,0,0,0,1,0
1224,Hotel Hazarduari,22.610069,88.404060,0,0,0,0,0,0,0,...,4.0,6,2621,353.09,4.12,0,0,0,1,0
1225,Collection O 207 Beck Bagan Ballygunge,22.538570,88.362839,1,0,0,0,0,0,1,...,4.0,5,1361,573.32,4.10,0,0,0,1,0
1226,Mira Guest House,22.543240,88.356361,1,0,0,0,0,0,0,...,4.0,6,2822,578.03,4.12,0,0,0,1,0


In [306]:
df_norm = df.copy(deep=True)
df_norm['Latitude'] = (df_norm['Latitude'] - min(df_norm['Latitude']))/(max(df_norm['Latitude']) - min(df_norm['Latitude']))
df_norm['Longitude'] = (df_norm['Longitude'] - min(df_norm['Longitude']))/(max(df_norm['Longitude']) - min(df_norm['Longitude']))
df_norm['Rating'] = (df_norm['Rating'] - min(df_norm['Rating']))/(max(df_norm['Rating']) - min(df_norm['Rating']))
df_norm['Number of reviews'] = (df_norm['Number of reviews'] - min(df_norm['Number of reviews']))/(max(df_norm['Number of reviews']) - min(df_norm['Number of reviews']))
df_norm['Price'] = (df_norm['Price'] - min(df_norm['Price']))/(max(df_norm['Price']) - min(df_norm['Price']))
df_norm

,Name,Latitude,Longitude,Air conditioning,Housekeeping,Private balcony,Private bathrooms,VIP room facilities,Complimentary toiletries,Room service,...,Rating,Number of reviews,Price,Distance Score,Review Score,Bangalore,Chennai,Delhi,Kolkata,Mumbai
0,Lemon Tree Premier-Mumbai International Airport,0.411798,0.006360,1,0,0,0,0,0,1,...,0.875,0.042076,0.147742,163.06,8.42,0,0,0,0,1
1,The Fern - Goregaon,0.415071,0.004768,1,0,0,0,0,0,1,...,0.875,0.085036,0.132959,134.39,9.27,0,0,0,0,1
2,"Taj Lands End, Mumbai",0.407920,0.002363,1,1,0,1,1,1,1,...,0.875,0.522474,0.215863,208.03,9.50,0,0,0,0,1
3,Grand Hyatt Mumbai Hotel & Residences,0.409922,0.004386,1,1,0,1,1,1,1,...,0.875,0.519934,0.222693,201.62,9.50,0,0,0,0,1
4,"Trident, Bandra Kurla, Mumbai",0.409298,0.005430,1,1,0,0,1,0,1,...,0.875,0.509332,0.197752,189.10,9.50,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1223,Backpackers Park,0.619690,0.988821,0,0,0,0,0,0,0,...,0.750,0.000994,0.014480,574.08,4.20,0,0,0,1,0
1224,Hotel Hazarduari,0.623070,0.992138,0,0,0,0,0,0,0,...,0.750,0.000552,0.093192,353.09,4.12,0,0,0,1,0
1225,Collection O 207 Beck Bagan Ballygunge,0.618757,0.989520,1,0,0,0,0,0,1,...,0.750,0.000442,0.038729,573.32,4.10,0,0,0,1,0
1226,Mira Guest House,0.619038,0.989109,1,0,0,0,0,0,0,...,0.750,0.000552,0.101880,578.03,4.12,0,0,0,1,0


In [307]:
X = df_norm.drop(['Name', 'Price'], axis = 1)
X = np.array(X)
X_o = df.drop(['Name', 'Price'], axis = 1)
X_o = np.array(X_o)

In [308]:
y = np.array(df_norm['Price'])
y_o = np.array(df['Price'])

In [289]:
Model = RandomForestRegressor()
Rfe = RFE(Model, n_features_to_select = 50)
Fit = Rfe.fit(X,y)
selected = Fit.support_
selected

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True, False,  True,
        True,  True,  True,  True,  True,  True,  True,  True, False,
       False,  True,  True,  True,  True,  True,  True,  True])

In [290]:
tbd = []
l = len(selected)
for i in range(l):
    if selected[i]==False:
        tbd.append(i)
        
tbd

[34, 44, 45]

In [278]:
X = np.delete(X, tbd, 1)
X

array([[4.11797533e-01, 6.35952310e-03, 1.00000000e+00, ...,
        4.20762010e-02, 1.63060000e+02, 8.42000000e+00],
       [4.15071030e-01, 4.76830985e-03, 1.00000000e+00, ...,
        8.50358918e-02, 1.34390000e+02, 9.27000000e+00],
       [4.07919713e-01, 2.36259312e-03, 1.00000000e+00, ...,
        5.22473771e-01, 2.08030000e+02, 9.50000000e+00],
       ...,
       [6.18756542e-01, 9.89519990e-01, 1.00000000e+00, ...,
        4.41744892e-04, 5.73320000e+02, 4.10000000e+00],
       [6.19038222e-01, 9.89108620e-01, 1.00000000e+00, ...,
        5.52181115e-04, 5.78030000e+02, 4.12000000e+00],
       [6.22805695e-01, 9.92585159e-01, 1.00000000e+00, ...,
        1.98785202e-03, 3.49100000e+02, 3.87000000e+00]])

In [325]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=53)
# for random state = 42 and number of features = 25 the results are good
# also for random state = 40 and number of features = 50, test_size = 0.2 in both cases
# random_state = 40
# random_state = 59 for scores (or 18 which is less)
# random_state = 53 for all cities together

In [326]:
reg = LinearRegression().fit(X_train,y_train)
reg_o = LinearRegression().fit(X_o,y_o)

In [327]:
print(reg.score(X_test,y_test))
print(reg_o.score(X_o, y_o))

0.39643066874366106
0.34268357291530627


In [328]:
rfr = RandomForestRegressor()
rfr_model = rfr.fit(X_train,y_train)

In [329]:
print("test score =", rfr_model.score(X_test,y_test))
print("train score =", rfr_model.score(X_train,y_train))

test score = 0.5105155911346207
train score = 0.8966235147219446


In [ ]:
MX = 0
for j in range(53):
    X = df_norm.drop(['Name', 'Price'], axis = 1)
    X = np.array(X)
    y = np.array(df_norm['Price'])
    mx = 0
    Model = RandomForestRegressor()
    Rfe = RFE(Model, n_features_to_select = j+1)
    Fit = Rfe.fit(X,y)
    selected = Fit.support_
    tbd = []
    l = len(selected)
    for i in range(l):
        if selected[i]==False:
            tbd.append(i)
    X = np.delete(X, tbd, 1)
    for i in range(100):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=i)
        rfr = RandomForestRegressor()
        rfr_model = rfr.fit(X_train,y_train)
        s = rfr_model.score(X_test, y_test)
        if s>mx:
            mx = s
            rs = i
    print(j+1, rs, mx)
    if mx > MX:
        MX = mx
        RS = rs
        NF = j+1
print(NF, RS, MX)